In [9]:
import torch

# Load the tensors
x_tensor_filtered = torch.load('/content/X_tensor_aftershocks_filtered.pt')
y_tensor_filtered = torch.load('/content/y_tensor_aftershocks_filtered.pt')

x_tensor_filtered_1 = torch.load('/content/X_tensor_pacific_belt_label1.pt')
y_tensor_filtered_1 = torch.load('/content/y_tensor_pacific_belt_label1.pt')

# Print the shapes
print(f"x_tensor_filtered shape: {x_tensor_filtered.shape}")
print(f"y_tensor_filtered shape: {y_tensor_filtered.shape}")

print(f"x_tensor_filtered_1 shape: {x_tensor_filtered_1.shape}")
print(f"y_tensor_filtered_1 shape: {y_tensor_filtered_1.shape}")

# Determine the length of the smaller dataset
length_1 = len(x_tensor_filtered_1)

# Randomly sample from the larger dataset
indices = torch.randperm(len(x_tensor_filtered))[:length_1]
x_tensor_filtered_sampled = x_tensor_filtered[indices]
y_tensor_filtered_sampled = y_tensor_filtered[indices]

# Concatenate the two datasets
x_combined = torch.cat((x_tensor_filtered_sampled, x_tensor_filtered_1), dim=0)
y_combined = torch.cat((y_tensor_filtered_sampled, y_tensor_filtered_1), dim=0)

# Print the shapes of the combined datasets
print(f"x_combined shape: {x_combined.shape}")
print(f"y_combined shape: {y_combined.shape}")

mean = x_combined.mean(dim=(0, 1))  # Mean for each feature across all samples and time steps
std = x_combined.std(dim=(0, 1))    # Std dev for each feature across all samples and time steps

# Normalize
x_combined_normalized = (x_combined - mean[None, None, :]) / std[None, None, :]


x_tensor_filtered shape: torch.Size([16234, 29, 6])
y_tensor_filtered shape: torch.Size([16234])
x_tensor_filtered_1 shape: torch.Size([1043, 29, 6])
y_tensor_filtered_1 shape: torch.Size([1043])
x_combined shape: torch.Size([2086, 29, 6])
y_combined shape: torch.Size([2086])


<ipython-input-9-c3b08e72cec0>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  x_tensor_filtered = torch.load('/content/X_tensor_aftershocks_filtered.pt')
<ipython-input-9-

In [10]:
from torch.utils.data import TensorDataset, DataLoader, random_split

# Assuming X_tensor_filtered and Y_tensor_filtered are already loaded
X = x_combined_normalized
Y = y_combined

# Create dataset from tensors
dataset = TensorDataset(X, Y)

# Splitting the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
batch_size = 64  # You can adjust the batch size
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [11]:
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])  # Take the last time step
        return out

# Initialize the LSTM model
input_dim = 6  # number of features
hidden_dim = 64
num_layers = 2
output_dim = 1  # Output dimension (regression)

model = LSTMModel(input_dim, hidden_dim, num_layers, output_dim)


In [12]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_loss = 0
            for inputs, labels in val_loader:
                outputs = model(inputs.float())
                val_loss += criterion(outputs.squeeze(), labels.float()).item()

        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

# Train the model
num_epochs = 10  # Number of epochs
train_model(model, train_loader, val_loader, num_epochs)


Epoch 1, Loss: 0.2674, Val Loss: 0.2432
Epoch 2, Loss: 0.1832, Val Loss: 0.1965
Epoch 3, Loss: 0.0847, Val Loss: 0.0600
Epoch 4, Loss: 0.0128, Val Loss: 0.0186
Epoch 5, Loss: 0.0065, Val Loss: 0.0144
Epoch 6, Loss: 0.0626, Val Loss: 0.0063
Epoch 7, Loss: 0.0002, Val Loss: 0.0037
Epoch 8, Loss: 0.0009, Val Loss: 0.0025
Epoch 9, Loss: 0.0003, Val Loss: 0.0021
Epoch 10, Loss: 0.0000, Val Loss: 0.0011


In [14]:
import torch
import torch.nn as nn

class BinaryLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(BinaryLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # Output dimension is 1 for binary classification
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation to output probabilities

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

# Initialize the LSTM model for binary classification
input_dim = 6  # Number of input features
hidden_dim = 64  # Hidden layers dimension
num_layers = 2  # Number of LSTM layers

model = BinaryLSTMModel(input_dim, hidden_dim, num_layers)

import torch.optim as optim

# Loss and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

def train_model(model, train_loader, val_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.float())
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            val_loss = 0
            for inputs, labels in val_loader:
                outputs = model(inputs.float())
                val_loss += criterion(outputs.squeeze(), labels.float()).item()

        print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}, Val Loss: {val_loss/len(val_loader):.4f}')

# Training the model

num_epochs = 10  # You can adjust the number of epochs
train_model(model, train_loader, val_loader, num_epochs)


Epoch 1, Loss: 0.4408, Val Loss: 0.6014
Epoch 2, Loss: 0.2613, Val Loss: 0.3032
Epoch 3, Loss: 0.0691, Val Loss: 0.1128
Epoch 4, Loss: 0.0248, Val Loss: 0.0484
Epoch 5, Loss: 0.0053, Val Loss: 0.0446
Epoch 6, Loss: 0.0048, Val Loss: 0.0234
Epoch 7, Loss: 0.0028, Val Loss: 0.0149
Epoch 8, Loss: 0.0018, Val Loss: 0.0075
Epoch 9, Loss: 0.0019, Val Loss: 0.0031
Epoch 10, Loss: 0.0011, Val Loss: 0.0053


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix


import numpy as np

# Define your LSTM model
class BinaryLSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(BinaryLSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # Output dimension is 1 for binary classification
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation to output probabilities

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

# Initialize the model
input_dim = 6  # Number of input features per earthquake
hidden_dim = 64  # Hidden layers dimension
num_layers = 2  # Number of LSTM layers

model = BinaryLSTMModel(input_dim, hidden_dim, num_layers)


# Define loss and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer








# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(x_combined_normalized, y_combined, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Train the model
def train_model(model, train_loader, val_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:

                outputs = model(inputs)
                val_loss += criterion(outputs.squeeze(), labels).item()
                all_preds.append(outputs.cpu().numpy())
                all_labels.append(labels.cpu().numpy())

        val_loss /= len(val_loader)
        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        # Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
        all_preds_binary = (all_preds > 0.5).astype(int)

        # Calculate and print the confusion matrix
        cm = confusion_matrix(all_labels, all_preds_binary)
        print(f'Confusion Matrix for Epoch {epoch+1}:')
        print(cm)

        # Compute ROC AUC score
        auc = roc_auc_score(all_labels, all_preds)

        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}, ROC AUC: {auc:.4f}')

# Train the model
num_epochs = 10  # You can adjust the number of epochs
train_model(model, train_loader, val_loader, num_epochs)


<ipython-input-21-53720dd92937>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
<ipython-input-21-53720dd92937>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32))


Confusion Matrix for Epoch 1:
[[163  50]
 [ 67 138]]
Epoch 1, Loss: 0.6820, Val Loss: 0.6349, ROC AUC: 0.8109
Confusion Matrix for Epoch 2:
[[180  33]
 [ 19 186]]
Epoch 2, Loss: 0.5001, Val Loss: 0.3481, ROC AUC: 0.9510
Confusion Matrix for Epoch 3:
[[207   6]
 [  3 202]]
Epoch 3, Loss: 0.1940, Val Loss: 0.0872, ROC AUC: 0.9967
Confusion Matrix for Epoch 4:
[[210   3]
 [  3 202]]
Epoch 4, Loss: 0.0775, Val Loss: 0.0582, ROC AUC: 0.9991
Confusion Matrix for Epoch 5:
[[210   3]
 [  0 205]]
Epoch 5, Loss: 0.0552, Val Loss: 0.0322, ROC AUC: 0.9997
Confusion Matrix for Epoch 6:
[[210   3]
 [  0 205]]
Epoch 6, Loss: 0.0297, Val Loss: 0.0284, ROC AUC: 1.0000
Confusion Matrix for Epoch 7:
[[213   0]
 [  0 205]]
Epoch 7, Loss: 0.0667, Val Loss: 0.0074, ROC AUC: 1.0000
Confusion Matrix for Epoch 8:
[[211   2]
 [  0 205]]
Epoch 8, Loss: 0.0677, Val Loss: 0.0316, ROC AUC: 1.0000
Confusion Matrix for Epoch 9:
[[213   0]
 [  0 205]]
Epoch 9, Loss: 0.0148, Val Loss: 0.0037, ROC AUC: 1.0000
Confusion 

In [23]:
# Define a GRU model for binary classification
class BinaryGRUModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(BinaryGRUModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 1)  # Output dimension is 1 for binary classification
        self.sigmoid = nn.Sigmoid()  # Sigmoid activation to output probabilities

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)

        # Forward propagate GRU
        out, _ = self.gru(x, h0)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        out = self.sigmoid(out)
        return out

# Initialize the GRU model
gru_model = BinaryGRUModel(input_dim, hidden_dim, num_layers)

# Define loss and optimizer for GRU model
gru_optimizer = optim.Adam(gru_model.parameters(), lr=0.001)  # Adam optimizer for GRU model

# Reuse the `train_model` function
def train_gru_model(model, train_loader, val_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:

            gru_optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            gru_optimizer.step()
            total_loss += loss.item()

        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:

                outputs = model(inputs)
                val_loss += criterion(outputs.squeeze(), labels).item()
                all_preds.append(outputs.cpu().numpy())
                all_labels.append(labels.cpu().numpy())

        val_loss /= len(val_loader)
        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        # Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
        all_preds_binary = (all_preds > 0.5).astype(int)

        # Calculate and print the confusion matrix
        cm = confusion_matrix(all_labels, all_preds_binary)
        print(f'Confusion Matrix for Epoch {epoch+1}:')
        print(cm)

        # Compute ROC AUC score
        auc = roc_auc_score(all_labels, all_preds)

        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Val Loss: {val_loss:.4f}, ROC AUC: {auc:.4f}')

# Train the GRU model
num_epochs = 10  # Adjust as needed
train_gru_model(gru_model, train_loader, val_loader, num_epochs)


Confusion Matrix for Epoch 1:
[[167  46]
 [ 51 154]]
Epoch 1, Loss: 0.6512, Val Loss: 0.5217, ROC AUC: 0.8388
Confusion Matrix for Epoch 2:
[[204   9]
 [  2 203]]
Epoch 2, Loss: 0.3238, Val Loss: 0.0809, ROC AUC: 0.9988
Confusion Matrix for Epoch 3:
[[210   3]
 [  0 205]]
Epoch 3, Loss: 0.0581, Val Loss: 0.0313, ROC AUC: 0.9998
Confusion Matrix for Epoch 4:
[[212   1]
 [  0 205]]
Epoch 4, Loss: 0.0264, Val Loss: 0.0107, ROC AUC: 1.0000
Confusion Matrix for Epoch 5:
[[213   0]
 [  0 205]]
Epoch 5, Loss: 0.0138, Val Loss: 0.0028, ROC AUC: 1.0000
Confusion Matrix for Epoch 6:
[[213   0]
 [  0 205]]
Epoch 6, Loss: 0.0095, Val Loss: 0.0015, ROC AUC: 1.0000
Confusion Matrix for Epoch 7:
[[213   0]
 [  0 205]]
Epoch 7, Loss: 0.0042, Val Loss: 0.0010, ROC AUC: 1.0000
Confusion Matrix for Epoch 8:
[[213   0]
 [  0 205]]
Epoch 8, Loss: 0.0015, Val Loss: 0.0007, ROC AUC: 1.0000
Confusion Matrix for Epoch 9:
[[213   0]
 [  0 205]]
Epoch 9, Loss: 0.0013, Val Loss: 0.0006, ROC AUC: 1.0000
Confusion 

In [25]:
# Define a CNN model for binary classification
import torch.nn.functional as F
class BinaryCNNModel(nn.Module):
    def __init__(self, input_dim, num_channels, kernel_size):
        super(BinaryCNNModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=num_channels, kernel_size=kernel_size)
        self.conv2 = nn.Conv1d(in_channels=num_channels, out_channels=num_channels*2, kernel_size=kernel_size)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(320, 64)  # Adjust based on actual flattened length
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape to [batch_size, num_features, num_earthquakes]
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)  # Flatten the output
        x = F.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x


# Initialize the CNN model
input_dim = 6  # Number of features per earthquake
num_channels = 32  # Number of output channels for the first convolution layer
kernel_size = 3  # Kernel size for the convolution layers

cnn_model = BinaryCNNModel(input_dim, num_channels, kernel_size)


# Define loss and optimizer for CNN model
cnn_optimizer = optim.Adagrad(cnn_model.parameters(), lr=0.0001)  # Adam optimizer for CNN model

# Reuse the `train_model` function
def train_cnn_model(model, train_loader, val_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:

            cnn_optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            cnn_optimizer.step()
            total_loss += loss.item()

        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:

                outputs = model(inputs)
                val_loss += criterion(outputs.squeeze(), labels).item()
                all_preds.append(outputs.cpu().numpy())
                all_labels.append(labels.cpu().numpy())

        val_loss /= len(val_loader)
        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        # Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
        all_preds_binary = (all_preds > 0.5).astype(int)

        # Calculate and print the confusion matrix
        cm = confusion_matrix(all_labels, all_preds_binary)


        # Compute ROC AUC score
        auc = roc_auc_score(all_labels, all_preds)

        if (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}, Val Loss: {val_loss:.4f}, ROC AUC: {auc:.4f}')
                print(f'Confusion Matrix for Epoch {epoch+1}:')
                print(cm)
# Train the CNN model
num_epochs = 100  # Adjust as needed
train_cnn_model(cnn_model, train_loader, val_loader, num_epochs)


Epoch 10, Loss: 0.6625, Val Loss: 0.6653, ROC AUC: 0.8651
Confusion Matrix for Epoch 10:
[[  0 213]
 [  0 205]]
Epoch 20, Loss: 0.6463, Val Loss: 0.6531, ROC AUC: 0.8834
Confusion Matrix for Epoch 20:
[[  0 213]
 [  0 205]]
Epoch 30, Loss: 0.6368, Val Loss: 0.6417, ROC AUC: 0.8937
Confusion Matrix for Epoch 30:
[[  6 207]
 [  0 205]]
Epoch 40, Loss: 0.6201, Val Loss: 0.6303, ROC AUC: 0.9007
Confusion Matrix for Epoch 40:
[[ 21 192]
 [  0 205]]
Epoch 50, Loss: 0.6151, Val Loss: 0.6184, ROC AUC: 0.9058
Confusion Matrix for Epoch 50:
[[ 49 164]
 [  1 204]]
Epoch 60, Loss: 0.6022, Val Loss: 0.6064, ROC AUC: 0.9092
Confusion Matrix for Epoch 60:
[[ 66 147]
 [  1 204]]
Epoch 70, Loss: 0.5901, Val Loss: 0.5941, ROC AUC: 0.9125
Confusion Matrix for Epoch 70:
[[ 86 127]
 [  1 204]]
Epoch 80, Loss: 0.5753, Val Loss: 0.5819, ROC AUC: 0.9154
Confusion Matrix for Epoch 80:
[[ 98 115]
 [  2 203]]
Epoch 90, Loss: 0.5589, Val Loss: 0.5697, ROC AUC: 0.9184
Confusion Matrix for Epoch 90:
[[113 100]
 [  

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Define a CNN model with dilation for binary classification
class BinaryCNNModel(nn.Module):
    def __init__(self, input_dim, num_channels, kernel_size, dilation_rate=1):
        super(BinaryCNNModel, self).__init__()
        # Adding dilation to convolutional layers
        self.conv1 = nn.Conv1d(in_channels=input_dim, out_channels=num_channels,
                               kernel_size=kernel_size, dilation=dilation_rate)
        self.conv2 = nn.Conv1d(in_channels=num_channels, out_channels=num_channels * 2,
                               kernel_size=kernel_size, dilation=dilation_rate)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(64 * 4, 64)  # Adjusted based on the flattened size
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = x.permute(0, 2, 1)  # Change shape to [batch_size, num_features, num_earthquakes]
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        x = x.view(x.size(0), -1)  # Flatten the output
        x = F.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x




# Initialize the CNN model with dilation
input_dim = 6  # Number of features per earthquake
num_channels = 32  # Number of output channels for the first convolution layer
kernel_size = 3  # Kernel size for the convolution layers
dilation_rate = 2  # Dilation rate for the convolution layers

cnn_model = BinaryCNNModel(input_dim, num_channels, kernel_size, dilation_rate)
# cnn_model = cnn_model.to('cuda')  # Move model to GPU

# Define loss and optimizer for CNN model
criterion = nn.BCELoss()
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.0001)  # Adam optimizer for CNN model

# Reuse the `train_model` function
def train_cnn_model(model, train_loader, val_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:
            # inputs, labels = inputs.to('cuda'), labels.to('cuda')
            cnn_optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            cnn_optimizer.step()
            total_loss += loss.item()

        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for inputs, labels in val_loader:
                # inputs, labels = inputs.to('cuda'), labels.to('cuda')
                outputs = model(inputs)
                val_loss += criterion(outputs.squeeze(), labels).item()
                all_preds.append(outputs.cpu().numpy())
                all_labels.append(labels.cpu().numpy())

        val_loss /= len(val_loader)
        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)

        # Convert probabilities to binary predictions (0 or 1) using a threshold of 0.5
        all_preds_binary = (all_preds > 0.5).astype(int)

        # Calculate and print the confusion matrix
        cm = confusion_matrix(all_labels, all_preds_binary)

        # Compute ROC AUC score
        auc = roc_auc_score(all_labels, all_preds)

        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch + 1}, Loss: {total_loss / len(train_loader):.4f}, Val Loss: {val_loss:.4f}, ROC AUC: {auc:.4f}')
            print(f'Confusion Matrix for Epoch {epoch+1}:')
            print(cm)

# Train the CNN model
num_epochs = 100  # Adjust as needed
train_cnn_model(cnn_model, train_loader, val_loader, num_epochs)


Epoch 10, Loss: 0.3279, Val Loss: 0.2967, ROC AUC: 0.9675
Confusion Matrix for Epoch 10:
[[193  20]
 [ 16 189]]
Epoch 20, Loss: 0.0916, Val Loss: 0.0918, ROC AUC: 0.9952
Confusion Matrix for Epoch 20:
[[207   6]
 [  2 203]]
Epoch 30, Loss: 0.0502, Val Loss: 0.0549, ROC AUC: 0.9987
Confusion Matrix for Epoch 30:
[[209   4]
 [  2 203]]
Epoch 40, Loss: 0.0297, Val Loss: 0.0325, ROC AUC: 0.9996
Confusion Matrix for Epoch 40:
[[210   3]
 [  2 203]]
Epoch 50, Loss: 0.0182, Val Loss: 0.0239, ROC AUC: 0.9998
Confusion Matrix for Epoch 50:
[[211   2]
 [  2 203]]
Epoch 60, Loss: 0.0135, Val Loss: 0.0189, ROC AUC: 0.9999
Confusion Matrix for Epoch 60:
[[211   2]
 [  0 205]]
Epoch 70, Loss: 0.0072, Val Loss: 0.0144, ROC AUC: 0.9999
Confusion Matrix for Epoch 70:
[[211   2]
 [  0 205]]
Epoch 80, Loss: 0.0068, Val Loss: 0.0118, ROC AUC: 1.0000
Confusion Matrix for Epoch 80:
[[212   1]
 [  0 205]]
Epoch 90, Loss: 0.0032, Val Loss: 0.0096, ROC AUC: 1.0000
Confusion Matrix for Epoch 90:
[[212   1]
 [  

In [32]:
import torch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import numpy as np


X = x_combined_normalized.reshape(x_combined_normalized.size(0), -1)  # Flatten sequences
Y = y_combined
# Split the resampled data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(
    n_estimators=100,            # Number of trees in the forest
    max_depth=100000,                # Maximum depth of each tree
    min_samples_split=5,         # Minimum number of samples required to split an internal node
    min_samples_leaf=3,          # Minimum number of samples required to be at a leaf node
    max_features='sqrt',         # Number of features to consider when looking for the best split
    random_state=42              # For reproducibility
)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred_proba = rf_model.predict_proba(X_val)[:, 1]  # Get probabilities for the positive class
y_pred = rf_model.predict(X_val)

# Compute ROC AUC and Accuracy
roc_auc = roc_auc_score(y_val, y_pred_proba)
accuracy = accuracy_score(y_val, y_pred)

print(f'Random Forest ROC AUC: {roc_auc:.4f}')
print(f'Random Forest Accuracy: {accuracy:.4f}')

# Compute Confusion Matrix
cm = confusion_matrix(y_val, y_pred)
print(cm)



Random Forest ROC AUC: 0.9605
Random Forest Accuracy: 0.9043
[[192  21]
 [ 19 186]]
